In [3]:
import pandas as pd
import pyshorteners
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

## Cleaning

In [4]:
anuncios = pd.read_csv("data/Anuncios-2024_09_30-11_17.xlsx - Anúncios (1).csv")
categories = pd.read_csv("data/lista-categorias.csv")
tabela_produtos = pd.read_csv("data/Tabela de Produtos - Página25.csv")
file_path = 'data/Fichas_tecnicas-2024_09_30-11_36.xlsx'

In [5]:
print(anuncios.info())
print(categories.info())
print(tabela_produtos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518 entries, 0 to 517
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   ITEM_ID                      515 non-null    object
 1   VARIATION_ID                 101 non-null    object
 2   TITLE                        514 non-null    object
 3   VARIATIONS                   514 non-null    object
 4   QUANTITY                     516 non-null    object
 5   CHANNEL                      514 non-null    object
 6   MARKETPLACE_PRICE            516 non-null    object
 7   MSHOPS_PRICE                 514 non-null    object
 8   MSHOPS_PRICE_SYNC            515 non-null    object
 9   CURRENCY_ID                  515 non-null    object
 10  SHIPPING_METHOD_MARKETPLACE  517 non-null    object
 11  SHIPPING_METHOD_MSHOPS       514 non-null    object
 12  LISTING_TYPE                 515 non-null    object
 13  FEE_PER_SALE_MARKETPLACE     515 no

In [6]:
tabela_produtos.fillna("-")
tabela_produtos.columns = tabela_produtos.columns.str.strip()
tabela_produtos= tabela_produtos.drop_duplicates(subset='ad_code', keep='first').fillna("-")


In [7]:
anuncios = anuncios.drop(index=[0, 1, 2, 3,4])
anuncios['STATUS'] = anuncios['STATUS'].str.strip()  # Remove espaços em branco no início e no fim
anuncios = anuncios.drop_duplicates(subset='ITEM_ID', keep='first')
anuncios =anuncios.drop(columns=['VARIATIONS' ,'VARIATION_ID'])
anuncios['STATUS'].value_counts()

Inativo    225
Ativo      188
Name: STATUS, dtype: int64

In [8]:
with_sku = tabela_produtos['ad_code']
with_sku_list = with_sku.tolist()

In [9]:
# DataFrame com os valores desejados
with_sku_products = anuncios[anuncios['ITEM_ID'].isin(with_sku_list)].rename(columns={'ITEM_ID':'ad_code','TITLE':'name',})
with_sku_products_df = with_sku_products[['name', 'ad_code','MARKETPLACE_PRICE',
       'MSHOPS_PRICE']]

In [10]:
# DataFrame com os valores desejados
without_sku_products = anuncios[~anuncios['ITEM_ID'].isin(with_sku_list)].rename(columns={'ITEM_ID':'ad_code','TITLE':'name',})
without_sku_products_df = without_sku_products[['name', 'ad_code','MARKETPLACE_PRICE',
       'MSHOPS_PRICE']]

In [11]:
without_sku_products.columns

Index(['ad_code', 'name', 'QUANTITY', 'CHANNEL', 'MARKETPLACE_PRICE',
       'MSHOPS_PRICE', 'MSHOPS_PRICE_SYNC', 'CURRENCY_ID',
       'SHIPPING_METHOD_MARKETPLACE', 'SHIPPING_METHOD_MSHOPS', 'LISTING_TYPE',
       'FEE_PER_SALE_MARKETPLACE', 'FEE_PER_SALE_MSHOPS', 'STATUS'],
      dtype='object')

In [12]:
without_sku_products_df

,name,ad_code,MARKETPLACE_PRICE,MSHOPS_PRICE
5,Sephiroth: Final Fantasy 7 Vii Remake Deluxe,MLB5089609766,2400,2350
6,Biohazard/resident Evil Village Edição De Cole...,MLB5089599494,1600,1550
7,Playstation 2 Fat,MLB5089595680,2100,2050
9,Split Pad Nintendo Switch Hori Split Pad Pokém...,MLB5089547950,600,550
11,Biohazard Anniversary Package Ps3,MLB5089440760,900,850
...,...,...,...,...
506,The Last Of Us 1 Ps3 Original Lacrado,MLB3431147473,800,800
507,South Park The Stick Of Truth Ps3 Original,MLB3431108807,150,120
508,Controle Dualsense Ps5 Lebron James Limited Ed...,MLB3427156693,1450,1400
510,Fone Starfield Limited Edition Xbox - Headset,MLB3427148359,1550,1350


In [13]:
with_sku_products_df

,name,ad_code,MARKETPLACE_PRICE,MSHOPS_PRICE
22,Elden Ring Shadow Of The Erdtree Collector's E...,MLB5064795618,3850,3599
27,Final Fantasy Xv Ultimate Collector's Edition ...,MLB4946967936,3990,3685
30,Sony Playstation 2 Fat Mobile Suit Z-gundam Ty...,MLB4946947218,4399,3990
35,Destiny 2 Ace Of Spades Miniatura Bungie Oficial,MLB4935929968,585,500
37,Onimusha Ps2 Trilogy Limited Edition Only Japa...,MLB4935547820,850,785
...,...,...,...,...
512,Dead Or Alive Ultimate Lacrado- Xbox Classico ...,MLB3422968763,650,550
513,Pocket Watch Limited Edition Super Mário Bros ...,MLB3420123395,400,350
514,Radar Dragon Ball Capsule Corp. Item Japonês I...,MLB3419939681,800,700
515,Mass Effect 2 Collectors Edition Xbox 360,MLB3411844229,900,800


In [14]:
tabela_produtos

,name,sku,universal_code,ad_code,condition
0,Elden Ring Shadow Of The Erdtree CE - Ps5,001-2409-0001,4949776471033,MLB5064795618,A
1,Onimusha Ps2 Trilogy LE Only Japan - Original,001-2409-0002,001-2409-0002,MLB4935547820,B
2,Kingdom Hearts Iii - Deluxe Edition Sony Ps4,001-2409-0003,662248921808,MLB4934685992,B+
3,Cuphead Edição Super Deluxe Para Switch - Only...,001-2409-0004,001-2409-0004,MLB4934496432,C
4,Jogo One Piece Grand Battle 2 Original Japones...,001-2409-0005,001-2409-0005,MLB4913347908,A
...,...,...,...,...,...
213,Controle Sem Fio Nintendo Switch Pro Controlle...,004-2409-0011,004-2409-0011,MLB3759261025,-
214,Metal Gear Solid 4 - Guns Of The Patriots - Ps3,001-2409-0160,001-2409-0160,MLB4834723702,-
215,Biohazard Code: Veronica - Ps2 + Cd Demo Devil...,001-2409-0161,001-2409-0161,MLB3472157767,-
216,Winning Eleven 8 Original Japones - Os,001-2409-0162,001-2409-0162,MLB4392438468,-


In [15]:
merged_df = pd.merge(with_sku_products_df, tabela_produtos, on=['ad_code','name'])
merged_df

,name,ad_code,MARKETPLACE_PRICE,MSHOPS_PRICE,sku,universal_code,condition
0,Destiny 2 Ace Of Spades Miniatura Bungie Oficial,MLB4935929968,585,500,002-2409-0002,18030100,A
1,Kingdom Hearts Iii - Deluxe Edition Sony Ps4,MLB4934685992,550,485,001-2409-0003,662248921808,B+
2,Cuphead Edição Super Deluxe Para Switch - Only...,MLB4934496432,699,650,001-2409-0004,001-2409-0004,C
3,Batman Furyu Justice League Special Figure Imp...,MLB4934481442,1150,1050,002-2409-0003,002-2409-0003,B+
4,Biohazard Revelations Unveiled Edition - Ps3,MLB4908972924,199,170,001-2409-0006,4976219050197,A
...,...,...,...,...,...,...,...
146,Street Fighter Vs. Tekken Edição Especial - X-360,MLB3472136175,850,799,001-2409-0108,4976219042383,A+
147,Soul Calibur - Dreamcast Japonês,MLB3470046061,210,190,001-2409-0109,001-2409-0109,A
148,Metal Gear Solid Hd Collection Ps3,MLB3422984935,220,190,001-2409-0111,83717202417,A
149,Dead Or Alive Ultimate Lacrado- Xbox Classico ...,MLB3422968763,650,550,001-2409-0112,4960677070071,Z


In [16]:
df_concatenated = pd.concat([merged_df, without_sku_products_df], ignore_index=True, sort=False)
df_concatenated

,name,ad_code,MARKETPLACE_PRICE,MSHOPS_PRICE,sku,universal_code,condition
0,Destiny 2 Ace Of Spades Miniatura Bungie Oficial,MLB4935929968,585,500,002-2409-0002,18030100,A
1,Kingdom Hearts Iii - Deluxe Edition Sony Ps4,MLB4934685992,550,485,001-2409-0003,662248921808,B+
2,Cuphead Edição Super Deluxe Para Switch - Only...,MLB4934496432,699,650,001-2409-0004,001-2409-0004,C
3,Batman Furyu Justice League Special Figure Imp...,MLB4934481442,1150,1050,002-2409-0003,002-2409-0003,B+
4,Biohazard Revelations Unveiled Edition - Ps3,MLB4908972924,199,170,001-2409-0006,4976219050197,A
...,...,...,...,...,...,...,...
347,The Last Of Us 1 Ps3 Original Lacrado,MLB3431147473,800,800,NaN,NaN,NaN
348,South Park The Stick Of Truth Ps3 Original,MLB3431108807,150,120,NaN,NaN,NaN
349,Controle Dualsense Ps5 Lebron James Limited Ed...,MLB3427156693,1450,1400,NaN,NaN,NaN
350,Fone Starfield Limited Edition Xbox - Headset,MLB3427148359,1550,1350,NaN,NaN,NaN


In [17]:
planilhas = {
    'Videogames': 'Jogos',
    'Figuras de acao': 'Figuras de ação',
    'Consoles de videogames': 'Consoles de videogames',
    'Gamepads e joysticks': 'Gamepads e joysticks',
    'Figuras interativas para vid...': 'Figuras interativas para vídeo',
    'Consoles de videogames  vide...': 'Consoles de videogames (vídeo)',
    'Relogios de pulso': 'Relógios de pulso',
    'Skins para consoles e contro...': 'Skins para consoles e controles',
    'Camisetas': 'Camisetas',
    'Livros': 'Livros',
    'Veiculos em miniatura': 'Veículos em miniatura',
    'Camisas de futebol': 'Camisas de futebol',
    'Cameras e barras sensoras pa...': 'Câmeras e barras sensores',
    'Controles remotos multimidia...': 'Controles remotos multimídia',
    'Tenis': 'Tênis',
    'Brinquedos e hobbies': 'Brinquedos e hobbies',
    'Produtos nao classificados': 'Produtos não classificados',
    'Controles remotos para repro...': 'Controles remotos para reprodução',
    'Oculos de realidade virtual': 'Óculos de realidade virtual',
    'Relogios de parede': 'Relógios de parede'
}

# Lista para armazenar os DataFrames filtrados
df_list = []

# Ler e processar cada planilha
for planilha, categoria in planilhas.items():
    df = pd.read_excel(file_path, sheet_name=planilha).drop(index=[0, 1, 2])
    df_filtrado = df[['ID', 'TITLE']].copy()  # Seleciona apenas as colunas 'ID' e 'TITLE'
    df_filtrado['categoria'] = categoria  # Adiciona a coluna 'Categoria'
    df_list.append(df_filtrado)

# Concatenar todos os DataFrames em um único DataFrame
produtos = pd.concat(df_list, ignore_index=True)

# Gerar códigos individuais numéricos para cada categoria com prefixo 'C'
categoria_codigos = {categoria: f'C{str(i + 1).zfill(3)}'
                     for i, categoria in enumerate(planilhas.values())}

# Adicionar a coluna 'Codigo' ao DataFrame
produtos['code'] = produtos['categoria'].map(categoria_codigos)
produtos.rename(columns={'ID': 'ad_code'}, inplace=True)
produtos = produtos.drop_duplicates(subset='ad_code', keep='first')
cat_codes = produtos[['ad_code', 'code',	'categoria']]

In [18]:
cat_codes

,ad_code,code,categoria
0,MLB5089599494,C001,Jogos
1,MLB5089440760,C001,Jogos
2,MLB5088632140,C001,Jogos
3,MLB5088474592,C001,Jogos
4,MLB5064795618,C001,Jogos
...,...,...,...
403,MLB4105805592,C016,Brinquedos e hobbies
404,MLB3419939681,C017,Produtos não classificados
405,MLB4836757574,C018,Controles remotos para reprodução
406,MLB3515862487,C019,Óculos de realidade virtual


In [19]:
cat_update = pd.merge(df_concatenated, cat_codes, on='ad_code')
cat_update

,name,ad_code,MARKETPLACE_PRICE,MSHOPS_PRICE,sku,universal_code,condition,code,categoria
0,Destiny 2 Ace Of Spades Miniatura Bungie Oficial,MLB4935929968,585,500,002-2409-0002,18030100,A,C002,Figuras de ação
1,Kingdom Hearts Iii - Deluxe Edition Sony Ps4,MLB4934685992,550,485,001-2409-0003,662248921808,B+,C001,Jogos
2,Cuphead Edição Super Deluxe Para Switch - Only...,MLB4934496432,699,650,001-2409-0004,001-2409-0004,C,C001,Jogos
3,Batman Furyu Justice League Special Figure Imp...,MLB4934481442,1150,1050,002-2409-0003,002-2409-0003,B+,C002,Figuras de ação
4,Biohazard Revelations Unveiled Edition - Ps3,MLB4908972924,199,170,001-2409-0006,4976219050197,A,C001,Jogos
...,...,...,...,...,...,...,...,...,...
342,The Last Of Us 1 Ps3 Original Lacrado,MLB3431147473,800,800,NaN,NaN,NaN,C001,Jogos
343,South Park The Stick Of Truth Ps3 Original,MLB3431108807,150,120,NaN,NaN,NaN,C001,Jogos
344,Controle Dualsense Ps5 Lebron James Limited Ed...,MLB3427156693,1450,1400,NaN,NaN,NaN,C004,Gamepads e joysticks
345,Fone Starfield Limited Edition Xbox - Headset,MLB3427148359,1550,1350,NaN,NaN,NaN,C006,Consoles de videogames (vídeo)


In [20]:
# Função para gerar SKU
def generate_sku(category_prefix, counter):
    if len(category_prefix) != 4 or not category_prefix[1:].isdigit():
        raise ValueError("O prefixo da categoria deve ter o formato correto (uma letra seguida por três números).")
    
    # Prefixo da categoria (parte numérica)
    category_number = category_prefix[1:]
    
    # Número sequencial formatado
    product_code = f"{counter:04}"
    
    # Retornar o SKU final no formato "1234-0001"
    return f"{category_number}-{product_code}"

# Dicionário para contar SKUs por categoria
sku_counts = {}

def get_next_sku_number(category_prefix):
    if category_prefix not in sku_counts:
        sku_counts[category_prefix] = 1
    else:
        sku_counts[category_prefix] += 1
    return sku_counts[category_prefix]

# Copiar DataFrame original para evitar modificações acidentais
produtos_cadastratos = cat_update.copy()

# Iterar sobre o DataFrame e preencher os SKUs faltantes
for idx, row in produtos_cadastratos.iterrows():
    if pd.isna(row['sku']):  # Verifica se o SKU está nulo
        category = row['code']
        # Gera o SKU com base na categoria e número sequencial
        produtos_cadastratos.at[idx, 'sku'] = generate_sku(category, get_next_sku_number(category))


In [21]:
produtos_cadastratos

,name,ad_code,MARKETPLACE_PRICE,MSHOPS_PRICE,sku,universal_code,condition,code,categoria
0,Destiny 2 Ace Of Spades Miniatura Bungie Oficial,MLB4935929968,585,500,002-2409-0002,18030100,A,C002,Figuras de ação
1,Kingdom Hearts Iii - Deluxe Edition Sony Ps4,MLB4934685992,550,485,001-2409-0003,662248921808,B+,C001,Jogos
2,Cuphead Edição Super Deluxe Para Switch - Only...,MLB4934496432,699,650,001-2409-0004,001-2409-0004,C,C001,Jogos
3,Batman Furyu Justice League Special Figure Imp...,MLB4934481442,1150,1050,002-2409-0003,002-2409-0003,B+,C002,Figuras de ação
4,Biohazard Revelations Unveiled Edition - Ps3,MLB4908972924,199,170,001-2409-0006,4976219050197,A,C001,Jogos
...,...,...,...,...,...,...,...,...,...
342,The Last Of Us 1 Ps3 Original Lacrado,MLB3431147473,800,800,001-0070,NaN,NaN,C001,Jogos
343,South Park The Stick Of Truth Ps3 Original,MLB3431108807,150,120,001-0071,NaN,NaN,C001,Jogos
344,Controle Dualsense Ps5 Lebron James Limited Ed...,MLB3427156693,1450,1400,004-0023,NaN,NaN,C004,Gamepads e joysticks
345,Fone Starfield Limited Edition Xbox - Headset,MLB3427148359,1550,1350,006-0009,NaN,NaN,C006,Consoles de videogames (vídeo)


In [22]:
produtos_cadastratos.to_excel('produtos_cadastratos.xlsx', index=False)

### Pesquisador de links

In [23]:
lista_principais = produtos_cadastratos['name']
lista_principais

0       Destiny 2 Ace Of Spades Miniatura Bungie Oficial
1           Kingdom Hearts Iii - Deluxe Edition Sony Ps4
2      Cuphead Edição Super Deluxe Para Switch - Only...
3      Batman Furyu Justice League Special Figure Imp...
4           Biohazard Revelations Unveiled Edition - Ps3
                             ...                        
342                The Last Of Us 1 Ps3 Original Lacrado
343           South Park The Stick Of Truth Ps3 Original
344    Controle Dualsense Ps5 Lebron James Limited Ed...
345       Fone Starfield Limited Edition Xbox - Headset 
346    Biohazard 2 Collectors Edition - Resident Evil...
Name: name, Length: 347, dtype: object

In [ ]:
# Inicializa o encurtador de URLs
s = pyshorteners.Shortener()

# Função para realizar a busca, coletar o código universal e o link
def search_and_collect_universal_codes(names):
    # Configurar o WebDriver
    driver = webdriver.Chrome()

    # URL do Google
    url = 'https://www.google.com'
    driver.get(url)

    # Criar um DataFrame vazio para armazenar os resultados
    results_df = pd.DataFrame(columns=['name', 'link', 'universal_code', 'qr_code_link'])

    for name in names:
        # Esperar a página carregar
        time.sleep(2)

        # Encontrar o campo de pesquisa
        search_box = driver.find_element(By.NAME, 'q')

        # Limpar o campo de pesquisa
        search_box.clear()

        # Inserir o termo de pesquisa focado em código universal e enviar
        search_term = f'{name} código universal'  # Modifique a busca para "código universal"
        search_box.send_keys(search_term)
        search_box.send_keys(Keys.RETURN)

        # Esperar os resultados carregarem
        time.sleep(3)

        try:
            # Acessar o primeiro link do resultado
            first_result = driver.find_element(By.CSS_SELECTOR, 'h3')
            first_result.click()

            # Esperar a nova página carregar
            time.sleep(3)

            # Obter a URL da página acessada
            current_url = driver.current_url

            # Encurtar a URL
            short_url = s.tinyurl.short(current_url)

            # Agora, você pode tentar encontrar o código universal na página
            # Isso pode variar muito de página para página
            page_source = driver.page_source

            # Exemplo simples de como capturar um possível código universal (UPC, ISBN, EAN)
            # Esse regex é um exemplo, pode ser ajustado conforme a necessidade
            import re
            universal_code = re.search(r'\b(?:UPC|EAN|ISBN)[\s:]*([\d-]+)\b', page_source)
            universal_code = universal_code.group(1) if universal_code else 'Código não encontrado'

            # Criar um novo DataFrame com o resultado atual, incluindo o código universal e o link encurtado
            new_row = pd.DataFrame({
                'name': [name], 
                'link': [current_url], 
                'universal_code': [universal_code], 
                'qr_code_link': [short_url]
            })

            # Concatenar o novo DataFrame com o existente
            results_df = pd.concat([results_df, new_row], ignore_index=True)

        except Exception as e:
            print(f"Erro ao buscar o nome {name}: {e}")
            # Em caso de erro, salvar uma entrada com erro no link
            new_row = pd.DataFrame({
                'name': [name], 
                'link': ['Erro ao acessar o link'], 
                'universal_code': ['Erro'], 
                'qr_code_link': ['Erro']
            })
            results_df = pd.concat([results_df, new_row], ignore_index=True)

        # Voltar à página do Google para a próxima pesquisa
        driver.get(url)

    # Fechar o WebDriver
    driver.quit()

    return results_df

# Exemplo de uso
names = lista_principais  # Substitua pela sua lista de nomes
df_universal_codes = search_and_collect_universal_codes(names)

# # Salvar o DataFrame em um arquivo CSV
# df_universal_codes.to_csv('resultados_universal_codes.csv', index=False)

df_universal_codes
